In [ ]:
!pip install portalocker==1.0


In [ ]:
!python -m pip install open3d==0.9


In [ ]:
import json
from glob import glob
import math
import os

import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import azureml.core
from azureml.core import Workspace
from azureml.core.dataset import Dataset

# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

### Get access to workspace and use one of databases for visualisation. Using 'anon-pcd-7k' here as an example.

In [ ]:
workspace = Workspace.from_config()
workspace
dataset = Dataset.get_by_name(workspace, name='anon-pcd-7k')
type(dataset)

### Mount the data to your local compute machine

In [ ]:
# This takes up to a minute

with dataset.mount() as mount_context:
    # list top level mounted files and folders in the dataset
    print(os.listdir(mount_context.mount_point))

In [ ]:
print(mount_context.mount_point) #get the location of the mountpoint for your machine

In [ ]:
mount_context = dataset.mount()
mount_context.start()  # this will mount the file streams
print(mount_context.mount_point)


### get all the rgb images for one of the folder

In [ ]:
mount_point = mount_context.mount_point + '/qrcode'

In [ ]:
current_folder = '1583438052-nxew87bcv0'
working_dir = mount_point + '/' + current_folder + '/measure/1566562694355/'  # Replace last number with something existing
images_dir = working_dir + 'rgb/'
rgb_files = glob(images_dir + '*.jpg')
len(rgb_files)
# TODO refactor

In [ ]:
# print(mount_point)
# ! ls {mount_point}
# ! ls "{mount_point}/1583438052-nxew87bcv0/measure/1566562694355/rgb"

In [ ]:
top_files = rgb_files[0:15]

In [ ]:
# plotting the one of the image from the list 
im = Image.open(rgb_files[0])
plt.imshow(im);

### Plot all the images of folder containing the frames of a video and save them in a grid images

In [ ]:
# Config:
result_grid_filename = 'grid.jpg'
result_figsize_resolution = 80 # 1 = 100px

# images_list = os.listdir(images_dir)
images_count = len(top_files)
# print('Images: ', images_list)
print('Images count: ', images_count)

# Calculate the grid size:
grid_size = math.ceil(math.sqrt(images_count))

In [ ]:
# Create plt plot:
fig, axes = plt.subplots(grid_size, grid_size, figsize=(result_figsize_resolution, result_figsize_resolution))

current_file_number = 0
for image_filename in top_files:
    print(image_filename)
    x_position = current_file_number % grid_size
    y_position = current_file_number // grid_size
#     plt_image = plt.imread(images_dir +images_list[current_file_number])
    plt_image = plt.imread(image_filename)
    axes[x_position, y_position].imshow(plt_image)
#     print((current_file_number + 1), '/', images_count, ': ', image_filename)

    current_file_number += 1

plt.subplots_adjust(left=0.0, right=1.0, bottom=0.0, top=1.0)
plt.savefig(result_grid_filename)

In [ ]:
grid = Image.open('grid.jpg')
plt.imshow(grid)

### Get the label for the corresponding video capture

In [ ]:
json_file = mount_point+'/'+current_folder+'/targets.json'
with open(json_file) as f:
    data = json.load(f)
print(data)

### Get the point cloud file for the video capture

In [ ]:
pointcloud_files = working_dir + 'pc/'
pcd_files = glob(pointcloud_files + '*.pcd')
pcd_files, len(pcd_files)

In [ ]:
## visualise the one of the point cloud files
import open3d as o3
from open3d import JVisualizer

fragment = o3.io.read_point_cloud(pcd_files[0])
visualizer = JVisualizer()
visualizer.add_geometry(fragment)
visualizer.show()